In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
train = pd.read_csv('/kaggle/input/cs-challenge/training_set.csv', index_col="ID")

In [3]:
train = train.dropna(axis=1)

In [4]:
column_list = column_list = train.columns.to_list()
non_redundant_cols = [x for x in column_list if x.find('_max') == -1 and x.find('_min') == -1 and x.find('_c') == -1]

In [5]:
non_redundant_cols

['TARGET',
 'MAC_CODE',
 'Date_time',
 'Pitch_angle',
 'Pitch_angle_std',
 'Hub_temperature',
 'Hub_temperature_std',
 'Generator_speed',
 'Generator_speed_std',
 'Generator_bearing_1_temperature',
 'Generator_bearing_1_temperature_std',
 'Generator_bearing_2_temperature',
 'Generator_bearing_2_temperature_std',
 'Generator_stator_temperature',
 'Generator_stator_temperature_std',
 'Gearbox_bearing_1_temperature',
 'Gearbox_bearing_1_temperature_std',
 'Gearbox_bearing_2_temperature',
 'Gearbox_bearing_2_temperature_std',
 'Gearbox_oil_sump_temperature',
 'Gearbox_oil_sump_temperature_std',
 'Nacelle_angle',
 'Nacelle_angle_std',
 'Nacelle_temperature',
 'Nacelle_temperature_std',
 'Absolute_wind_direction',
 'Outdoor_temperature',
 'Outdoor_temperature_std',
 'Grid_frequency',
 'Grid_frequency_std',
 'Rotor_speed',
 'Rotor_speed_std',
 'Rotor_bearing_temperature',
 'Rotor_bearing_temperature_std']

In [6]:
#train_non_re = train[non_redundant_cols]
train_non_re=train

In [7]:
def ind_max(l):
    M=l[0]
    ind=0
    for i in range(1,len(l)):
        if l[i]>M:
            M=l[i]
            ind=i
    return ind

In [8]:
#squared_cols = []
pows=[-5+i/2 for i in range(21)]
pows.remove(0.0)
res=[]
#for col in non_redundant_cols:
for col in column_list:
    if col != 'MAC_CODE':
        corr=[]
        for p in pows:
            if (p%1==0 or not any(train[col]<0)) and (p>0 or not any(train[col]==0)):                
                    corr.append(abs(train_non_re['TARGET'].corr(train_non_re[col]**p)))
            else:
                corr.append(0)
        p=pows[ind_max(corr)]
        res.append(p)
        train_non_re[col] = np.power(train_non_re[col], p)
res
            
            
            #cor1 = abs(train_non_re['TARGET'].corr(train_non_re[col]))
            #cor2 = abs(train_non_re['TARGET'].corr(train_non_re[col]**2))
            #if(cor2 > cor1):
                #train_non_re[col] = np.power(train_non_re[col], 2)
                #squared_cols.append(col)
        

[1.0,
 -0.5,
 1.0,
 1.0,
 1.0,
 0.5,
 4.0,
 4.0,
 4.0,
 3.0,
 5.0,
 4.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 0.5,
 5.0,
 5.0,
 5.0,
 0.5,
 5.0,
 5.0,
 5.0,
 0.5,
 5.0,
 5.0,
 5.0,
 0.5,
 5.0,
 5.0,
 5.0,
 0.5,
 3.0,
 2.0,
 3.0,
 0.5,
 5.0,
 3.5,
 3.5,
 0.5,
 3.0,
 2.0,
 3.0,
 0.5,
 5.0,
 2.0,
 2.0,
 2.0,
 1.0,
 5.0,
 5.0,
 5.0,
 1.0,
 5.0,
 4.0,
 5.0,
 2.0,
 1.0,
 1.0,
 1.0,
 0.5]

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


col_transformer = ColumnTransformer([
    ('MAC_CODE', OneHotEncoder(dtype='int'),['MAC_CODE'])],
    remainder = StandardScaler())

reg = RidgeCV(cv=5)

pipe = Pipeline([('col_transformer', col_transformer),('reg', reg)], verbose=True)


In [10]:
train_non_re

,TARGET,MAC_CODE,Date_time,Pitch_angle,Pitch_angle_min,Pitch_angle_max,Pitch_angle_std,Hub_temperature,Hub_temperature_min,Hub_temperature_max,...,Grid_frequency_max,Grid_frequency_std,Rotor_speed,Rotor_speed_min,Rotor_speed_max,Rotor_speed_std,Rotor_bearing_temperature,Rotor_bearing_temperature_min,Rotor_bearing_temperature_max,Rotor_bearing_temperature_std
ID,,,,,,,,,,,,,,,,,,,,,
3,-0.79100,WT3,0.577350,92.470001,92.470001,92.470001,0.000000,2401.000000,2401.000000,2401.000000,...,3.115637e+08,0.01,0.000000e+00,0.000000,0.000000e+00,0.0000,2.400000,2.400000,2.400000,0.000000
7,0.10000,WT3,0.377964,92.470001,92.470001,92.470001,0.000000,1296.000000,1296.000000,2176.119744,...,0.000000e+00,0.00,0.000000e+00,0.000000,0.000000e+00,0.0000,2.400000,2.400000,2.400000,0.000000
10,0.10000,WT3,0.316228,92.470001,92.470001,92.470001,0.000000,285.343070,256.000000,625.000000,...,0.000000e+00,0.00,0.000000e+00,0.000000,0.000000e+00,0.0000,2.400000,2.350000,2.400000,0.000000
13,-5.98300,WT3,0.038691,92.470001,92.470001,92.470001,0.000000,16.000000,81.000000,16.000000,...,3.134386e+08,0.01,0.000000e+00,0.000000,0.000000e+00,0.0000,-2.550000,-2.850000,-2.550000,0.316228
14,-2.67200,WT3,0.038662,92.470001,92.470001,92.470001,0.000000,1.262476,16.000000,1.000000,...,3.125000e+08,0.01,0.000000e+00,0.000000,0.000000e+00,0.0000,-2.370000,-2.600000,-2.300000,0.223607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617381,1147.01500,WT1,0.002518,-1.000000,-1.010000,-0.930000,0.100000,104976.000000,95515.858381,104976.000000,...,3.134386e+08,0.01,1.436640e+06,42878.854598,1.604134e+06,0.1369,35.889999,35.799999,36.200001,0.346410
617383,1119.93300,WT1,0.002518,-1.010000,-1.010000,-1.010000,0.000000,104742.914328,93147.568033,104976.000000,...,3.128126e+08,0.01,1.407373e+06,56098.917274,1.590369e+06,0.1156,35.810001,35.599998,35.849998,0.244949
617384,1223.82800,WT1,0.002518,-0.810000,-1.010000,1.490000,0.761577,104976.000000,99712.524374,104976.000000,...,3.128126e+08,0.01,1.436640e+06,60294.262291,1.604134e+06,0.0900,36.060001,35.799999,36.099998,0.316228


In [11]:
from sklearn.model_selection import train_test_split

Xtr, Xte, ytr,  yte = train_test_split(train_non_re.drop('TARGET', axis=1), train_non_re['TARGET'], test_size=0.2)

pipe.fit(Xtr,ytr)

[Pipeline] ... (step 1 of 2) Processing col_transformer, total=   1.0s
[Pipeline] ............... (step 2 of 2) Processing reg, total=   4.4s


Pipeline(steps=[('col_transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('MAC_CODE',
                                                  OneHotEncoder(dtype='int'),
                                                  ['MAC_CODE'])])),
                ('reg', RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=5))],
         verbose=True)

In [12]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(yte, pipe.predict(Xte))

62.2869350934978

In [13]:
pipe.fit(train_non_re.drop('TARGET', axis=1),train_non_re['TARGET'])

[Pipeline] ... (step 1 of 2) Processing col_transformer, total=   1.3s
[Pipeline] ............... (step 2 of 2) Processing reg, total=   5.3s


Pipeline(steps=[('col_transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('MAC_CODE',
                                                  OneHotEncoder(dtype='int'),
                                                  ['MAC_CODE'])])),
                ('reg', RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=5))],
         verbose=True)

In [14]:
test = pd.read_csv('/kaggle/input/cs-challenge/test_set.csv', index_col="ID")
#test = test[[x for x in non_redundant_cols if x != 'TARGET']]
test = test[[x for x in column_list if x != 'TARGET']]

for col in squared_cols:
    test[col] = np.power(test[col],2)

predict = pipe.predict(test)

NameError: name 'squared_cols' is not defined

In [15]:
test['TARGET'] = predict
test['TARGET'].to_csv("squared_ridge.csv")

NameError: name 'predict' is not defined